In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [4]:
from virny.datasets.data_loaders import StudentPerformancePortugueseDataset
from virny.utils.protected_groups_partitioning import create_test_protected_groups

In [5]:
def get_proportions(protected_groups, X_data):
    for col_name in protected_groups.keys():
        proportion = protected_groups[col_name].shape[0] / X_data.shape[0]
        print(f'{col_name}: {round(proportion, 3)}')


def get_base_rate(protected_groups, y_data):
    for col_name in protected_groups.keys():
        filtered_df = y_data.iloc[protected_groups[col_name].index].copy(deep=True)
        base_rate = filtered_df[filtered_df == 1].shape[0] / filtered_df.shape[0]
        print(f'{col_name}: {round(base_rate, 3)}')

    base_rate = y_data[y_data == 1].shape[0] / y_data.shape[0]
    print(f'overall: {round(base_rate, 3)}')

In [6]:
sensitive_attributes_dct = {'sex': 'F'}

In [7]:
data_loader = StudentPerformancePortugueseDataset()
data_loader.full_df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,class
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,1
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,1


In [8]:
data_loader.full_df.shape

(649, 33)

In [9]:
data_loader.full_df['sex'].value_counts()

F    383
M    266
Name: sex, dtype: int64

In [10]:
protected_groups = create_test_protected_groups(data_loader.X_data, data_loader.X_data, sensitive_attributes_dct)

In [11]:
for col_name in protected_groups.keys():
    print(f'{col_name}: {protected_groups[col_name].shape[0]}')

sex_priv: 266
sex_dis: 383


In [12]:
protected_groups.keys()

dict_keys(['sex_priv', 'sex_dis'])

In [13]:
get_proportions(protected_groups, data_loader.X_data)

sex_priv: 0.41
sex_dis: 0.59


In [14]:
get_base_rate(protected_groups, data_loader.y_data)

sex_priv: 0.812
sex_dis: 0.869
overall: 0.846


In [23]:
data_loader.y_data.iloc[protected_groups['male_priv'].index].head()

KeyError: 'male_priv'

In [ ]:
data_loader.y_data.iloc[protected_groups['male_priv'].index].dtype

In [ ]:
filtered_df = data_loader.y_data.iloc[protected_groups['male_priv'].index].copy(deep=True)
base_rate = filtered_df[filtered_df == 1].shape[0] / filtered_df.shape[0]

In [ ]:
base_rate

In [ ]:
filtered_df[filtered_df == 1].shape[0]

In [ ]:
filtered_df.shape[0]